In [26]:
import pandas as pd
import torch 
import torch.nn as nn
from torch.nn import functional as F

import numpy as np

In [27]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [28]:
import tiktoken

In [29]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-14 15:33:35--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-09-14 15:33:35 (24.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [30]:
filename = "/kaggle/working/input.txt"

text = None
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

In [31]:
print("Length of the text: {}".format(len(text)))

Length of the text: 1115394


In [63]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [64]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [65]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [66]:
stoi = {char: idx for idx, char in enumerate(chars)}
itos = {idx: char for idx, char in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda n: "".join(itos[i] for i in n)

In [67]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [103]:
enc = tiktoken.get_encoding('gpt2')
vocab_size = enc.n_vocab

In [104]:
data = torch.tensor(enc.encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([338025]) torch.int64


In [105]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([304222]) torch.Size([33803])


In [150]:
batch_size = 8
block_size = 128
max_iters = 5000
lr = 3e-4
eval_iters = 200
n_embed = 384
n_head = 6
n_layers = 6
dropout = 0.15

In [151]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch("train")
print('inputs: ')
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

inputs: 
torch.Size([8, 128])
tensor([[ 1049, 15967,    11,  ...,   339,   319,   262],
        [ 3440,    11,   198,  ...,    40, 12797,   262],
        [  314,   750,  1011,  ...,   198,  7120,  1751],
        ...,
        [ 1276,   307, 14263,  ...,   621, 10938,    13],
        [  273,   415, 19921,  ...,   373, 18988,    11],
        [   48,  8924,  1677,  ..., 28356,   736,   286]], device='cuda:0')
targets: 
torch.Size([8, 128])
tensor([[15967,    11,   198,  ...,   319,   262,   835],
        [   11,   198,  1532,  ..., 12797,   262, 21144],
        [  750,  1011,   262,  ...,  7120,  1751,   547],
        ...,
        [  307, 14263,   276,  ..., 10938,    13,   198],
        [  415, 19921,   307,  ..., 18988,    11,   198],
        [ 8924,  1677,    25,  ...,   736,   286,  1918]], device='cuda:0')


In [152]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [153]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v
        return out

In [154]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(out)
        return self.projection(out)

In [155]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)

In [156]:
class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa_heads = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        x = x + self.sa_heads(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [157]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super(BigramLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        
    def forward(self, src, tgt=None):
        B, T = src.shape
        
        tok_emb = self.embedding(src) # (B, T, C)
        pos_emb = self.position_embedding(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if tgt is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tgt = tgt.view(B*T)
            loss = F.cross_entropy(logits, tgt)

        return logits, loss
    
    def generate(self, src, max_length):
        for _ in range(max_length):
            src_cond = src[:, -block_size:]
            
            logits, _ = self(src_cond)
            
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            
            idx_next = torch.multinomial(probs, num_samples=1)
            src = torch.cat((src, idx_next), dim=1)
            
        return src
    
model = BigramLanguageModel().to(device)
logits, loss = model(xb, yb)
print(loss, logits.shape)

tensor(11.0118, device='cuda:0', grad_fn=<NllLossBackward0>) torch.Size([1024, 50257])


In [158]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [159]:
for iter in range(max_iters):
    xb, yb = get_batch('train')
    
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}; train_loss: {losses['train']}; val_loss: {losses['val']}")
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0; train_loss: 10.994235038757324; val_loss: 11.003198623657227
step: 200; train_loss: 5.455996513366699; val_loss: 5.717780590057373
step: 400; train_loss: 4.888713836669922; val_loss: 5.332345962524414
step: 600; train_loss: 4.616786956787109; val_loss: 5.077981948852539
step: 800; train_loss: 4.416253566741943; val_loss: 4.92218542098999
step: 1000; train_loss: 4.2369608879089355; val_loss: 4.829485893249512
step: 1200; train_loss: 4.085729598999023; val_loss: 4.810875415802002
step: 1400; train_loss: 3.9390952587127686; val_loss: 4.726984977722168
step: 1600; train_loss: 3.814293146133423; val_loss: 4.655821800231934
step: 1800; train_loss: 3.6938366889953613; val_loss: 4.703493595123291
step: 2000; train_loss: 3.6097042560577393; val_loss: 4.7000274658203125
step: 2200; train_loss: 3.5206191539764404; val_loss: 4.6884260177612305
step: 2400; train_loss: 3.4336087703704834; val_loss: 4.705157279968262
step: 2600; train_loss: 3.347733497619629; val_loss: 4.6924543380737305
ste

In [160]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(model.generate(context, 1000)[0].tolist()))

!

COMINIUS:
And welcome, they we have good cause in ourselves:
Go whip you, fed home; take up, with us; for though we shall cry
 contradiction, and we shall find talk with cursing.

JULIET:
'Tis very road, well plainly I have trod measure in regard
What is the night-faced day to chop night,
The blood o' the soldiers, the movingend here soundly urged,
To teeming foamingacles.
Signior's not thy name, oldCourage; one with one:
What say to me with thee? in Mantua,

PAGE: and harlot people should fain
That marks thee in thy blessed ill in thee, that word:
More cruel for any more may behall:
Thou rather, who for the purpose that death
For that: therefore, I say! time thou art the likeness:
As likely a pale
And yet, too careless, the meran when ever I see't.

CORIOLANUS:
So I, a son?
There is this constant.

Second Messenger:
Have you moved to took the city Corioli?

Second Messenger:
Come, said, you shall bear the leisure to London.
Moreover, fare you our own house,
will out a puissance may